In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('https://zenodo.org/record/7394851/files/MN-DS-news-classification.csv?download=1')

In [6]:
df.head()

,data_id,id,date,source,title,content,author,url,published,published_utc,collection_utc,category_level_1,category_level_2
0,1809,abcnews--2019-10-31--Virginia mom charged with...,2019-10-31,abcnews,Virginia mom charged with murder in 2-year-old...,The Virginia woman whose 2-year-old son was fo...,NaN,https://abcnews.go.com/US/wireStory/virginia-m...,"Thu, 31 Oct 2019 16:49:56 -0400",1572554996,1572559512,"crime, law and justice",crime
1,1980,abcnews--2019-11-07--2 escaped murder suspects...,2019-11-07,abcnews,2 escaped murder suspects arrested at US-Mexic...,Authorities are trying to determine if anyone ...,NaN,https://abcnews.go.com/US/wireStory/escaped-mu...,"Thu, 07 Nov 2019 00:13:12 -0500",1573103592,1573131986,"crime, law and justice",crime
2,1995,abcnews--2019-11-07--Family turns in escaped b...,2019-11-07,abcnews,"Family turns in escaped boy, 13, suspected in ...",A 13-year-old suspect in a double homicide who...,NaN,https://abcnews.go.com/US/wireStory/family-tur...,"Thu, 07 Nov 2019 07:39:54 -0500",1573130394,1573131982,"crime, law and justice",crime
3,2740,abcnews--2019-12-02--Mother charged with murde...,2019-12-02,abcnews,Mother charged with murder in deaths of 2 youn...,The mother of two young children found hanging...,NaN,https://abcnews.go.com/US/wireStory/mother-cha...,"Mon, 02 Dec 2019 11:30:59 -0500",1575304259,1575308811,"crime, law and justice",crime
4,7038,ageofautism--2019-04-12--Physician Father and ...,2019-04-12,ageofautism,"Physician, Father and Caretaker of 29 Year Old...","""One family member said Derek “can be violent ...",Age of Autism,http://feedproxy.google.com/~r/ageofautism/~3/...,2019-04-12 09:00:00+00:00,1555074000,1567543083,"crime, law and justice",crime


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10917 entries, 0 to 10916
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   data_id           10917 non-null  int64 
 1   id                10917 non-null  object
 2   date              10917 non-null  object
 3   source            10917 non-null  object
 4   title             10917 non-null  object
 5   content           10917 non-null  object
 6   author            7605 non-null   object
 7   url               10917 non-null  object
 8   published         10917 non-null  object
 9   published_utc     10917 non-null  int64 
 10  collection_utc    10917 non-null  int64 
 11  category_level_1  10917 non-null  object
 12  category_level_2  10917 non-null  object
dtypes: int64(3), object(10)
memory usage: 1.1+ MB


## Feature selection for this model
- As we are working with recommendation system, we only need the `title` , `content` , `category_level_1` and `category_level_2` .
- So we will drop other columns.

In [9]:
columns_to_be_dropped = ['data_id','id','source','date','author','url','published','published_utc','collection_utc']
df = df.drop(columns_to_be_dropped,axis=1)
df.head()

,title,content,category_level_1,category_level_2
0,Virginia mom charged with murder in 2-year-old...,The Virginia woman whose 2-year-old son was fo...,"crime, law and justice",crime
1,2 escaped murder suspects arrested at US-Mexic...,Authorities are trying to determine if anyone ...,"crime, law and justice",crime
2,"Family turns in escaped boy, 13, suspected in ...",A 13-year-old suspect in a double homicide who...,"crime, law and justice",crime
3,Mother charged with murder in deaths of 2 youn...,The mother of two young children found hanging...,"crime, law and justice",crime
4,"Physician, Father and Caretaker of 29 Year Old...","""One family member said Derek “can be violent ...","crime, law and justice",crime


In [7]:
df.nunique()

data_id             10491
id                  10491
date                  355
source                215
title               10357
content             10380
author               2999
url                 10365
published           10373
published_utc       10368
collection_utc       8712
category_level_1       17
category_level_2      109
dtype: int64